In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# Requests
import requests
from time import sleep
from tqdm import tqdm

# Para convertir de string a diccionario
import ast
import os
import json

import sys
sys.path.append("../")
import src.soporte_spotify as api

In [2]:
sp = api.load_credentials()
sp.current_user()

{'country': 'ES',
 'display_name': 'MrApple',
 'email': 'ruiperezgonzalo4@gmail.com',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/gonzaloruiperez'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/users/gonzaloruiperez',
 'id': 'gonzaloruiperez',
 'images': [],
 'product': 'premium',
 'type': 'user',
 'uri': 'spotify:user:gonzaloruiperez'}

# Verificar si nos han bloqueado
- Tenemos un limite diario y si no spotify devuelve 429, algo que con spotipy no se ve, asi que usamos requests

In [3]:
token = api.request_token()
playlist_id = "1IMxg7e2dwhawHaoBfWWIH" 
url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

headers = {
    "Authorization": f"Bearer {token}"
}

response = requests.get(url, headers=headers)


print(f"{response.status_code}, Detalles: {response.text}")
print(response.headers)


Token obtenido con éxito
200, Detalles: {"collaborative":false,"description":"","external_urls":{"spotify":"https://open.spotify.com/playlist/1IMxg7e2dwhawHaoBfWWIH"},"followers":{"href":null,"total":168},"href":"https://api.spotify.com/v1/playlists/1IMxg7e2dwhawHaoBfWWIH","id":"1IMxg7e2dwhawHaoBfWWIH","images":[{"height":null,"url":"https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da84475b6d364ac1a57b52370891","width":null}],"name":"Best Songs Ever","owner":{"display_name":"Daniele Carrera","external_urls":{"spotify":"https://open.spotify.com/user/31dgcpvufdzr5u7ytwdycm7irwaq"},"href":"https://api.spotify.com/v1/users/31dgcpvufdzr5u7ytwdycm7irwaq","id":"31dgcpvufdzr5u7ytwdycm7irwaq","type":"user","uri":"spotify:user:31dgcpvufdzr5u7ytwdycm7irwaq"},"primary_color":null,"public":true,"snapshot_id":"AAAAeRoZiYlZR88EWDv9Mc2Jc9CcC31M","tracks":{"href":"https://api.spotify.com/v1/playlists/1IMxg7e2dwhawHaoBfWWIH/tracks?offset=0&limit=100","items":[{"added_at":"2024-12-29T18:27:24Z","add

In [4]:
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv',index_col=0)
primark = pd.read_csv("../datos/01 Spotify/02_followers_primark.csv",index_col=0)
nike = pd.read_csv("../datos/01 Spotify/03_followers_nike.csv",index_col=0)
adidas = pd.read_csv("../datos/01 Spotify/04_followers_adidas.csv",index_col=0)
hm = pd.read_csv("../datos/01 Spotify/05_followers_hm.csv",index_col=0)
pullbear = pd.read_csv("../datos/01 Spotify/06_followers_pull_and_bear.csv",index_col=0)
bershka = pd.read_csv("../datos/01 Spotify/07_followers_bershka.csv",index_col=0)
stradivarius = pd.read_csv("../datos/01 Spotify/08_followers_stradivarius.csv",index_col=0)
mango = pd.read_csv("../datos/01 Spotify/09_followers_mango.csv",index_col=0)
desigual = pd.read_csv("../datos/01 Spotify/10_followers_desigual.csv",index_col=0)

# Zara
### Obtener ID canciones
- Por cada id de playlist, sacaremos los tracks con su respectivo id de canción


In [5]:
zara.sample()

,brand,username,user_id,playlists
558,Zara,miimi_4,miimi_4,"{'シアターライフ': '0Yt7nwpfxbGzOVyuG9Ne0M', 'adore':..."


### Convertir columna playlists a diccionario
- utilizamos la librería ast para obtener el diccionario

In [6]:
zara["playlists"] = zara["playlists"].apply(ast.literal_eval)
type(zara["playlists"][0])

dict

### Sacamos a una lista todos los ids de playlists y otra con máximo 10
- Limitamos a 10 playlists para que usuarios con más no desbalanceen al resto!

In [7]:
playlist_ids = []
for ids in zara["playlists"]:
    playlist_ids.append(list(ids.values()))

zara["playlist_ids"] = playlist_ids
zara["playlist_ids_limited"] = zara["playlist_ids"].apply(lambda x: x[:10] if isinstance(x,list) else x)

### Obtener los artistas de cada usuario

In [9]:
lista_artistas = []
output_file = "../datos/01 Spotify/00_GuardadoTemporal.csv"

# Cargar progreso si existe un archivo previo
if os.path.exists(output_file):
    zara = pd.read_csv(output_file)
    zara["playlist_ids_limited"] = zara["playlist_ids_limited"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
    lista_artistas = zara["artistas"].tolist()
else:
    zara["artistas"] = None

# Procesar playlists limitadas
for idx, id_playlist in enumerate(tqdm(zara["playlist_ids_limited"], desc="Obteniendo artistas")):
    if pd.notna(zara.loc[idx, "artistas"]):
        continue
    id_playlist = list(id_playlist)

    # Solicitar token y obtener artistas
    token = api.request_token(silent=True)
    artistas = api.obtener_artistas(token, id_playlist)
    lista_artistas.append(artistas)
    
    
    # Guardar los artistas como JSON
    zara.loc[idx, "artistas"] = json.dumps(artistas)

    # Guardar el progreso después de cada iteración
    zara.to_csv(output_file, index=False)

Obteniendo artistas:  69%|██████▉   | 605/873 [00:20<00:01, 234.98it/s]

Rate limit alcanzado. Durmiendo por 13.20 segundos...


Obteniendo artistas:  70%|███████   | 615/873 [00:46<00:27,  9.41it/s] 

Error al procesar la playlist 45MDvUJ7xu29vwGORZfXJs. Código: 404


Obteniendo artistas:  77%|███████▋  | 670/873 [04:22<06:42,  1.98s/it]

...

Obteniendo artistas:  95%|█████████▌| 831/873 [16:14<02:50,  4.06s/it]

Rate limit alcanzado. Durmiendo por 8.78 segundos...
Rate limit alcanzado. Durmiendo por 9.62 segundos...
Rate limit alcanzado. Durmiendo por 11.90 segundos...
Rate limit alcanzado. Durmiendo por 12.13 segundos...
Rate limit alcanzado. Durmiendo por 13.56 segundos...


Obteniendo artistas:  96%|█████████▌| 836/873 [19:05<08:48, 14.29s/it]

Error al procesar la playlist 1XlfJ4ZsMWFLFpVZIiPNCP. Código: 404


Obteniendo artistas: 100%|██████████| 873/873 [21:03<00:00,  1.45s/it]


In [11]:
zara.to_csv("../datos/01 Spotify/01_followers_zara.csv",index=False)